# RAG Workbench - LlamaIndex Quickstart

In this notebook, we showcase how to use the **LastMile Tracing SDK** and **RAG Workbench** to manage, evaluate, and debug your LlamaIndex applications. With tracing automatically setup, you can easily debug your application using LastMile's RAG Workbench.

## Notebook Outline
* [Step 1: Install and Setup](#setup)
* [Step 2: Configure LastMile Instrumentor](#step2)
* [Step 3: Load Document](#step3)
* [Step 4: Create an Index and Query Engine](#step4)
* [Step 5: Query the Index](#step5)
* [Step 6: View Results in RAG Workbench UI](#step6)


<a name="setup"></a>
# Step 1: Install and Setup

First install the required packages.

In [ ]:
!pip install llama-index-embeddings-openai --upgrade
%pip install -q html2text llama-index pandas pyarrow tqdm
%pip install -q llama-index-readers-web
%pip install -q llama-index-callbacks-openinference
!pip install openai --upgrade
!pip install "tracing-auto-instrumentation[llama-index]" --upgrade

We also need the following API tokens/keys:

* **LastMile AI API Token:** Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* **OpenAI API Key:** Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

Run the code cell below after setting the keys either in **Google Colab Secrets** or in `.env` in your directory. Avoid inputting keys directly in the notebook.

In [ ]:
import os

try:
    # If running on Google Colab, use userdata to securely input keys
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    LASTMILE_API_TOKEN = userdata.get('LASTMILE_API_TOKEN')
except ModuleNotFoundError:
    # If running locally, load keys from .env file
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    LASTMILE_API_TOKEN = os.getenv('LASTMILE_API_TOKEN')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['LASTMILE_API_TOKEN'] = LASTMILE_API_TOKEN

<a name="step2"></a>

## Step 2: Configure LastMile Instrumentor

Next, we need to configure the LastMile Instrumentor which will auto-instrument tracing for your LlamaIndex application. This allows us to easily see the various steps in our RAG system in the RAG Workbench UI.

In [ ]:

import llama_index.core
from tracing_auto_instrumentation.llama_index import LlamaIndexCallbackHandler
from lastmile_eval.rag.debugger.common.types import RagFlowType

PROJECT_NAME = "LM-Tutorial"

cb_handler = LlamaIndexCallbackHandler(
    project_name=PROJECT_NAME,
)
tracer = cb_handler.get_tracer()
tracer.set_rag_flow_type(RagFlowType.INGESTION)
llama_index.core.global_handler = cb_handler

<a name="step3"></a>

# Step 3: Ingest Document


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader

@tracer.trace_function("Data Ingestion")
def ingest():
    documents = SimpleWebPageReader().load_data(["https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"])
    index = VectorStoreIndex.from_documents(documents)
    return index

index = ingest()

<a name="step4"></a>

# Step 4: Create an Index and Query Engine

In [ ]:
from lastmile_eval.rag.debugger.tracing import get_latest_ingestion_trace_id

# Transition to Query Mode
tracer.set_rag_flow_type(RagFlowType.QUERY)
tracer.ingestion_trace_id = get_latest_ingestion_trace_id(project_name=PROJECT_NAME)
query_engine = index.as_query_engine()

<a name="step5"></a>

# Step 5: Query the Index

In [ ]:
import textwrap

max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?"
]

for query in queries:
    response = query_engine.query(query)
    print("Query")
    print("=====")
    print(textwrap.fill(query, max_characters_per_line))
    print()
    print("Response")
    print("========")
    print(textwrap.fill(str(response), max_characters_per_line))
    print()

<a name="step6"></a>

# Step 6: View Results in RAG Workbench UI
You can view the results in the RAG Workbench UI. Here's how to access the UI:

1. Install the UI package by running the following command in your terminal:

    `pip install "lastmile-eval[ui]”`

2. Export your LASTMILE_API_TOKEN by running the following command in your terminal, replacing <your-api-token> with your actual API token:

    `export LASTMILE_API_TOKEN="<your-api-token>”`

3. Run the following command in your terminal:

    `rag-debug launch`

Once you've completed these steps, click on the url provided to open the RAG Workbench UI. 

1. Click the **Traces Tab**.
2. Select Project **'LlamaIndex-QuickStart'**. You can see the Trace logged in the UI.

<img width="850" alt="llama_overview" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/01c084be-01a6-4d77-a4e4-a59e00e55af7">



3. Click on a Trace to see the steps of our LlamaIndex application and other metadata.

<img width="850" alt="llama_trace_view" src="https://github.com/lastmile-ai/eval-cookbook/assets/141073967/0c5ed6b8-2668-4479-9b59-e5e8b1b0c338">

Now you can dive into specific steps and check out logs, parameters, and other aspects of your application!

